In [1]:
import pandas as pd
import nltk
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Embedding, Input,LSTM,GlobalMaxPool1D, Dropout
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.preprocessing.text import one_hot

In [13]:
true_df = pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\True.csv")
fake_df=pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\Fake.csv")

In [15]:
#Add label Column
true_df['label'] = 0
fake_df['label'] = 1
#true_df

In [17]:
#Combined Both Files 
combined_df = pd.concat([true_df, fake_df], ignore_index=True)
#combined_df

In [19]:
combined_df=combined_df.drop(['date'],axis=1)

In [21]:
combined_df['Text'] = combined_df['title'] + " " + combined_df['text'] + " " + combined_df['subject']
combined_df=combined_df.drop(['title','text','subject'],axis=1)

In [23]:
y = combined_df['label'].values
X = combined_df.drop(['label'],axis=1)

In [25]:
# configuration settingof LSTM (According to Research Paper)
voc_size = 10000
sent_length = 1000
Test_size = 0.3
batch_size = 120
EPOCHS = 30 # original 164 but Progress is not shown
Learning_rate = 0.0001
embedding_vector_features = 100

In [27]:
onehot_repr = [one_hot(words, voc_size) for words in X['Text']]

In [28]:
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size=Test_size, random_state=42)

In [33]:
model = Sequential()

# Add layers to the model
model.add(Input(shape=(sent_length,)))
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
optimizer = SGD(learning_rate=Learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 1000, 100)           │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,080,501 (4.12 MB)

 Trainable params: 1,080,501 (4.12 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=batch_size)

Epoch 1/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 615s 2s/step - accuracy: 0.5223 - loss: 0.6931
Epoch 2/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 620s 2s/step - accuracy: 0.5369 - loss: 0.6930
Epoch 3/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 634s 2s/step - accuracy: 0.5382 - loss: 0.6929
Epoch 4/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 630s 2s/step - accuracy: 0.5490 - loss: 0.6927
Epoch 5/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 621s 2s/step - accuracy: 0.5541 - loss: 0.6926
Epoch 6/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 632s 2s/step - accuracy: 0.5668 - loss: 0.6924
Epoch 7/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 638s 2s/step - accuracy: 0.5582 - loss: 0.6924
Epoch 8/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 645s 2s/step - accuracy: 0.5676 - loss: 0.6922
Epoch 9/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 639s 2s/step - accuracy: 0.5661 - loss: 0.6921
Epoch 10/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 703s 3s/step - accuracy: 0.5707 - loss: 0.6919
Epoch 11/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 882s 3s/step - accuracy: 0.5643 - loss: 0.6919
Epoch 12/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 

In [37]:
y_pred = model.predict(X_test)

421/421 ━━━━━━━━━━━━━━━━━━━━ 88s 208ms/step


In [39]:
y_pred[:5]

array([[0.50868535],
       [0.50777674],
       [0.50893474],
       [0.5041903 ],
       [0.5062786 ]], dtype=float32)

In [45]:
report = classification_report(y_test, y_pred.round(), zero_division=0)

In [47]:
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6474
           1       0.52      1.00      0.68      6996

    accuracy                           0.52     13470
   macro avg       0.26      0.50      0.34     13470
weighted avg       0.27      0.52      0.36     13470

